## Exploration of KernelSHAP with binary MNIST

**Function        : Exploration of KernelSHAP with binary MNIST**<br>
**Author          : Team DIANNA**<br>
**Contributor     :**<br>
**First Built     : 2021.11.24**<br>
**Last Update     : 2021.11.30**<br>
**Library         : os, numpy, matplotlib, tensorflow, keras, shap**<br>
**Description     : In this notebook we test XAI method KernelSHAP using trained binary MNIST model.**<br>
**Return Values   : Shapley scores**<br>
**Note**          : We use shap library, which is the original implementation by the author of "SHAP" paper, to perform KernelSHAP.<br>

In [1]:
# this is the code from https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [2]:
# prepare binary mnist dataset

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# check the shape of train and test sets
print("Before filtering")
print("training set shape", x_train.shape)
print("training set label shape", y_train.shape)
print("testing set shape", x_test.shape)
print("testing set label shape", y_test.shape)
# get all the images labelled "0" and "1" (binary filtering)
label_a = 0
label_b = 1
x_train_binary = x_train[(y_train == label_a) | (y_train == label_b),:]
y_train_binary = y_train[(y_train == label_a) | (y_train == label_b)]
x_test_binary = x_test[(y_test == label_a) | (y_test == label_b),:]
y_test_binary = y_test[(y_test == label_a) | (y_test == label_b)]
# check the shape of train and test sets after filtering
print("After filtering")
print("training set shape", x_train_binary.shape)
print("training set label shape", y_train_binary.shape)
print("testing set shape", x_test_binary.shape)
print("testing set label shape", y_test_binary.shape)

Before filtering
training set shape (60000, 28, 28)
training set label shape (60000,)
testing set shape (10000, 28, 28)
testing set label shape (10000,)
After filtering
training set shape (12665, 28, 28)
training set label shape (12665,)
testing set shape (2115, 28, 28)
testing set label shape (2115,)


In [3]:
# define basic parameters of training
batch_size = 128
num_classes = 2
epochs = 2

# preprocess training and testing sets
if K.image_data_format() == 'channels_first':
    x_train_binary = x_train_binary.reshape(x_train_binary.shape[0], 1, img_rows, img_cols)
    x_test_binary = x_test_binary.reshape(x_test_binary.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train_binary = x_train_binary.reshape(x_train_binary.shape[0], img_rows, img_cols, 1)
    x_test_binary = x_test_binary.reshape(x_test_binary.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train_binary = x_train_binary.astype('float32')
x_test_binary = x_test_binary.astype('float32')
x_train_binary /= 255
x_test_binary /= 255
print('x_train_binary shape:', x_train_binary.shape)
print(x_train_binary.shape[0], 'train samples')
print(x_test_binary.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train_binary = keras.utils.to_categorical(y_train_binary, num_classes)
y_test_binary = keras.utils.to_categorical(y_test_binary, num_classes)

x_train_binary shape: (12665, 28, 28, 1)
12665 train samples
2115 test samples


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

2021-11-30 14:00:06.199401: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-30 14:00:06.200067: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-30 14:00:06.202277: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model.fit(x_train_binary, y_train_binary,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_binary, y_test_binary))
score = model.evaluate(x_test_binary, y_test_binary, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2021-11-30 14:00:10.724355: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-30 14:00:10.730446: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2304005000 Hz


Epoch 1/2
99/99 [==============================] - 14s 134ms/step - loss: 0.6787 - accuracy: 0.6471 - val_loss: 0.6519 - val_accuracy: 0.9551
Epoch 2/2
99/99 [==============================] - 26s 263ms/step - loss: 0.6481 - accuracy: 0.8154 - val_loss: 0.6189 - val_accuracy: 0.9768
Test loss: 0.6189391613006592
Test accuracy: 0.9768321514129639


In [16]:
import shap
import numpy as np

# explain predictions of the model on three images
e = shap.KernelExplainer(model.predict, x_train_binary[:10,:,:,:])
shap_values = e.shap_values(x_test_binary[0,:,:,0], nsamples=50)
#shap_values = e.shap_values(x_test_binary[:1,:,:,:], nsamples=50)
#shap_values = e.shap_values(np.ones((1, img_rows * img_cols)), nsamples=50)

  0%|          | 0/28 [00:00<?, ?it/s]

ValueError: in user code:

    /home/yangliu/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/yangliu/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/yangliu/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/yangliu/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/yangliu/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/yangliu/miniconda3/lib/python3.8/site-packag